In [ ]:
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt
from scipy import signal

# ==========================================================
#   1) PREÁMBULO QPSK ROBUSTO (BARKER 13 ROTADO)
# ==========================================================

def generate_qpsk_preamble():
    # Barker 13: +1 +1 +1 +1 +1 -1 -1 +1 +1 -1 +1 -1 +1
    barker = np.array([1,1,1,1,1,-1,-1,1,1,-1,1,-1,1])

    # Convertimos a fase alternada 0°/90°
    phases = (barker > 0) * 0 + (barker < 0) * (np.pi/2)
    
    return phases  # array de 13 fases QPSK


# ==========================================================
#   2) MODULADOR QPSK (solo para el preámbulo)
# ==========================================================

def modulate_qpsk(phases, Tb, fs, fc):
    N = int(Tb * fs)
    total_samples = len(phases) * N
    t = np.linspace(0, len(phases)*Tb, total_samples, endpoint=False)

    signal_out = np.zeros(total_samples)

    for k, phi in enumerate(phases):
        start = k*N
        end   = start + N
        
        carrier = np.cos(2*np.pi*fc*t[start:end] + phi)
        signal_out[start:end] = carrier

    return t, signal_out


# ==========================================================
#   3) MODULADOR 8PSK (TUYO, SIN RC)
# ==========================================================

def modulate_8psk_data(bits, Tb, fs, fc):
    mapping = {
        "000": 0,
        "001": np.pi/4,
        "011": np.pi/2,
        "010": 3*np.pi/4,
        "110": np.pi,
        "111": 5*np.pi/4,
        "101": 3*np.pi/2,
        "100": 7*np.pi/4,
    }

    symbols = [bits[i:i+3] for i in range(0, len(bits), 3)]
    phases = np.array([mapping[s] for s in symbols])

    N = int(Tb * fs)
    total_samples = len(phases)*N
    t = np.linspace(0, len(phases)*Tb, total_samples, endpoint=False)

    s = np.zeros(total_samples)

    for k, phi in enumerate(phases):
        start = k*N
        end   = start+N
        carrier = np.cos(2*np.pi*fc*t[start:end] + phi)
        s[start:end] = carrier

    return t, s


# ==========================================================
#   4) CORRELACIÓN NORMALIZADA
# ==========================================================

def norm_corr(x, h):
    corr = signal.correlate(x, h, mode='valid')
    energia_x = np.sqrt(signal.convolve(x**2, np.ones(len(h)), mode='valid'))
    energia_h = np.sqrt(np.sum(h**2))
    return corr / (energia_x * energia_h + 1e-12)


# ==========================================================
#   5) DETECTOR DE PRIMER PICO
# ==========================================================

def detect_first_peak(ncc, thr=0.4):
    idxs = np.where(ncc > thr)[0]
    if len(idxs)==0:
        return None
    return idxs[0]


# ==========================================================
#   6) FUNCIÓN COMPLETA DE TRASMISIÓN+RECEPCIÓN
# ==========================================================

def record_and_receive_with_preamble(bits_8psk,
                                     Tb=0.01,
                                     fs=48000,
                                     fc=3000,
                                     max_duration=7):
    
    print("🎙 Grabando audio...")
    N = int(max_duration*fs)
    recording = sd.rec(N, samplerate=fs, channels=1, dtype='float32')
    sd.wait()
    rx = recording[:,0]
    print("✔ Grabado.")

    # ====== MODULAR PREÁMBULO QPSK ======
    preamble_phases = generate_qpsk_preamble()
    _, s_pre = modulate_qpsk(preamble_phases, Tb, fs, fc)

    # ====== MODULAR DATOS 8PSK ======
    _, s_data = modulate_8psk_data(bits_8psk, Tb, fs, fc)

    # ====== MODULAR POSTÁMBULO QPSK ======
    _, s_post = modulate_qpsk(preamble_phases, Tb, fs, fc)

    # Construimos plantilla de correlación del preámbulo
    h = s_pre / np.max(np.abs(s_pre))

    # Normalizar recepción
    rx_norm = rx / (np.max(np.abs(rx))+1e-12)

    # ====== DETECTAR INICIO ======
    print("🔍 Detección de inicio...")
    ncc_start = norm_corr(rx_norm, h)
    start_idx = detect_first_peak(ncc_start, thr=0.4)

    if start_idx is None:
        raise RuntimeError("❌ No se detectó preámbulo de inicio.")

    print(f"✔ Inicio detectado en {start_idx/fs:.3f} s")

    # ====== DETECTAR FIN ======
    print("🔍 Detección de fin...")
    search_region = rx_norm[start_idx + len(h):]
    ncc_end = norm_corr(search_region, h)
    end_rel = detect_first_peak(ncc_end, thr=0.4)

    if end_rel is None:
        raise RuntimeError("❌ No se detectó postámbulo.")

    end_idx = start_idx + len(h) + end_rel
    print(f"✔ Fin detectado en {end_idx/fs:.3f} s")

    # ====== EXTRAER DATOS ======
    msg_signal = rx[start_idx+len(h): end_idx]
    print(f"📦 Mensaje extraído: {len(msg_signal)/fs:.3f} s")

    return t, rx, msg_signal
